In [1]:
import cv2
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
import os
from PIL import Image, ImageFile
from torch.utils.data import Dataset
import torch
import torch.optim as optim
import torchvision.transforms as transsforms

from torchvision import transforms
from torch.optim import lr_scheduler

In [2]:
def load_ben_color(path, sigmaX=10):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #image = cv2.resize(image, (224, 224)) #224 224
    #image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
        
    return image



In [3]:
class PupilDatasetTrain(Dataset):

    def __init__(self, df, transform=None):

        self.data = df
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join('./dataset50/', self.data.loc[idx, 1] + '.jpg')
        image = load_ben_color(img_name)
        label = torch.tensor(train_csv.loc[idx,[2,3]].values.astype(np.float16))
        image = transforms.ToPILImage()(image)
        if self.transform:
            image = self.transform(image)
        return image, label

In [4]:
train_tf = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [5]:
train_csv = pd.read_csv('./target50_clustered.csv', header = None)
clusters = train_csv[4].values

In [6]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(train_csv, clusters,stratify = clusters, test_size = 0.2, random_state=2019)[0:2]
train_df, val_df = train_test_split(train_csv, clusters,stratify = clusters, test_size = 0.1, random_state=2019)[0:2]
train_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [7]:
trainset     = PupilDatasetTrain(train_df, transform =train_tf)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=8, shuffle=True, num_workers=0)
valset       = PupilDatasetTrain(val_df, transform   =train_tf)
val_loader   = torch.utils.data.DataLoader(valset, batch_size=8, shuffle=False, num_workers=0)

In [8]:
num_classes = 2
lr          = 1e-3
IMG_SIZE    = 224

In [9]:
from efficientnet_pytorch import EfficientNet
from torch.optim import Adam, SGD, RMSprop
import torch.nn.init as init
import torch
import torch.nn as nn

model = EfficientNet.from_pretrained('efficientnet-b0') 
in_features = model._fc.in_features
model._fc = nn.Linear(in_features, num_classes)
model.cuda()

Loaded pretrained weights for efficientnet-b0


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=

In [10]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
criterion = nn.MSELoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)

In [11]:
def train_model(epoch):
    model.train() 
        
    avg_loss = 0.
    optimizer.zero_grad()
    for idx, (imgs, labels) in enumerate(train_loader):
        imgs_train, labels_train = imgs.cuda(), labels.float().cuda()
        output_train = model(imgs_train)
        loss = criterion(output_train,labels_train)
        loss.backward()
        optimizer.step() 
        optimizer.zero_grad() 
        avg_loss += loss.item() / len(train_loader)
        
    return avg_loss

def test_model():
    
    avg_val_loss = 0.
    model.eval()
    with torch.no_grad():
        for idx, (imgs, labels) in enumerate(val_loader):
            imgs_vaild, labels_vaild = imgs.cuda(), labels.float().cuda()
            output_test = model(imgs_vaild)
            avg_val_loss += criterion(output_test, labels_vaild).item() / len(val_loader)
        
    return avg_val_loss

In [12]:
import time


best_avg_loss = 30000
n_epochs      = 30

for epoch in range(n_epochs):
    
    print('lr:', scheduler.get_lr()[0]) 
    start_time   = time.time()
    avg_loss     = train_model(epoch)
    avg_val_loss = test_model()
    elapsed_time = time.time() - start_time 
    print('Epoch {}/{} \t loss={:.4f} \t val_loss={:.4f} \t time={:.2f}s'.format(
        epoch + 1, n_epochs, avg_loss, avg_val_loss, elapsed_time))
    
    if avg_val_loss < best_avg_loss:
        best_avg_loss = avg_val_loss
        torch.save(model.state_dict(), './weight_best_b0_minusrez.pt')
    
    scheduler.step()



lr: 0.001
Epoch 1/30 	 loss=63526.5524 	 val_loss=29426.1074 	 time=153.27s
lr: 0.001
Epoch 2/30 	 loss=12559.2103 	 val_loss=13280.1873 	 time=154.31s
lr: 0.001
Epoch 3/30 	 loss=7928.0119 	 val_loss=10252.1799 	 time=153.81s
lr: 0.001
Epoch 4/30 	 loss=7774.7691 	 val_loss=9244.6953 	 time=153.14s
lr: 0.001
Epoch 5/30 	 loss=7747.4075 	 val_loss=10856.7468 	 time=153.55s
lr: 0.001
Epoch 6/30 	 loss=7718.1563 	 val_loss=9272.3128 	 time=153.97s
lr: 0.001
Epoch 7/30 	 loss=7706.0172 	 val_loss=9250.0143 	 time=154.34s
lr: 0.001
Epoch 8/30 	 loss=7673.5059 	 val_loss=9963.9175 	 time=156.22s
lr: 0.001
Epoch 9/30 	 loss=7602.8694 	 val_loss=9464.8962 	 time=160.34s
lr: 0.001
Epoch 10/30 	 loss=7591.0702 	 val_loss=9229.1196 	 time=245.17s
lr: 0.001
Epoch 11/30 	 loss=7603.3040 	 val_loss=9792.8073 	 time=187.69s
lr: 0.001
Epoch 12/30 	 loss=7663.4291 	 val_loss=9608.4643 	 time=203.90s
lr: 0.001
Epoch 13/30 	 loss=7604.4789 	 val_loss=9617.5581 	 time=175.84s
lr: 0.001
Epoch 14/30 	 loss